In [1]:
import cobra
import cobrakbase

cobrakbase 0.2.7


In [ ]:
is_transport_pts(modelseed_local.get_seed_reaction('rxn02005'))
is_transport_abc(modelseed_local.get_seed_reaction('rxn33882'))

In [3]:
%run ../../cobrakbase/cobrakbase/modelseed/transporters.py
import pandas as pd
tp = TransportersPipeline(modelseed_local, annotation_api)
print('permeases', len(tp.t_permeases))
print('t_pts', len(tp.t_pts))
print('t_abc', len(tp.t_abc))
print('t_others', len(tp.t_others))
#multiple match
#multiple match
#permeases 1401
#t_pts 35
#t_abc 1089
#t_others 1402

NameError: name 'modelseed_local' is not defined

In [ ]:
df_abc = pd.read_csv('/Users/fliu/workspace/jupyter/data/modelseed2/annotation_abc.tsv', '\t')
df_pts = pd.read_csv('/Users/fliu/workspace/jupyter/data/modelseed2/annotation_pts.tsv', '\t')

In [ ]:
missing_reaction, reaction_roles, role_reactions = tp.f1(df_pts, tp.t_pts, [], is_transport_pts)

In [ ]:
for rxn_id in reaction_roles:
    sn_to_roles = {}
    for role_str in reaction_roles[rxn_id]:
        sn = normalize_role(role_str)
        if not sn in sn_to_roles:
            sn_to_roles[sn] = set()
        sn_to_roles[sn].add(role_str)
    o = template.get_reaction(rxn_id + '_c')
    if not o == None:
        break

In [ ]:
complex_roles = o.get_complex_roles()
for complex_id in complex_roles:
    for role_id in complex_roles[complex_id]:
        role = template.get_role(role_id)
        sn = normalize_role(role['name'])
        verified = sn in sn_to_roles
        print(rxn_id, role_id, verified, role['name'])

In [ ]:
tp = TransportersPipeline(modelseed_local)
cpd_alias = tp.get_cpd_alias()
tp.report()

In [ ]:
tp.report(role_reactions)

### Annotation for David

In [ ]:
huge_trans_genes = pd.read_csv('/Users/fliu/workspace/jupyter/data/tnseq/transyt/refG_data_genes_list.txt', '\t', header=None)
huge_trans_genes = huge_trans_genes.set_index(0)

In [ ]:
genome_gene_function = {}
for genome_id, d in huge_trans_genes.iterrows():
    if not genome_id in genome_gene_function:
        genome_gene_function[genome_id] = {}
    if not pd.isna(d[1]):
        genes = d[1].split(';')
        for gene_id in genes:
            gfunctions = get_gene_function(gene_id, genome_id)
            genome_gene_function[genome_id][gene_id] = gfunctions
    break

In [ ]:
def get_gene_function(gene_id, genome_id, source = 'BETA_RAST'):
    node_gene = annotation_api.get_node('KBaseGene', gene_id + '@' + genome_id)
    if node_gene == None:
        logger.warning('gene not found %s %s', gene_id, genome_id)
        return None
    node_gene = annotation_api.matcher.get(node_gene.id)
    gfunctions = set()
    for rel in annotation_api.neo4j_graph.match((node_gene, ), r_type="has_annotation", ):
        if source in rel['function_source']:
            function = Neo4jAnnotationFunction(rel.end_node)
            gfunctions.add(function.value)
    return gfunctions

In [ ]:
data = {
    'genome_id' : [],
    'gene_id' : [],
    'function' : []
}
for genome_id in genome_gene_function:
    for gene_id in genome_gene_function[genome_id]:
        gfunction = genome_gene_function[genome_id][gene_id]
        gfstr = "?" if gfunction == None else ';'.join(gfunction)
        data['genome_id'].append(genome_id)
        data['gene_id'].append(gene_id)
        data['function'].append(gfstr)